In [1]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd

In [2]:
URL_lhr = "http://lhr.nu.edu.pk/faculty/"
r_lhr = requests.get(URL_lhr) 
print(r_lhr) 

soup_lhr = BeautifulSoup(r_lhr.content, 'html.parser') 

<Response [200]>


In [3]:
# Extracting the faculty member cards
faculty_lhr = soup_lhr.findAll('div', class_='facultyCard')

C:\Users\f-jaf\AppData\Local\Temp\ipykernel_19280\2217577771.py:2: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  faculty_lhr = soup_lhr.findAll('div', class_='facultyCard')


In [4]:
ID=[]
name=[]
email=[]
designation=[]
img=[]
highest_education=[]
HEC_approved=[]
dept=[]
extension=[]

In [5]:
# extracting ID , image , name , email, designation and if HEC Approved Supervisor
for f in faculty_lhr:
    
    try:
        if(f.find('p',class_='small').text.strip().split('\n')[2]):
            HEC_approved.append(True)
            
    except IndexError:
        HEC_approved.append(False)
    
    try:
        ID.append(int(f.find('a',class_='faculty-link')['href'].split('/')[-1]))
        
    except:
        ID.append(-1)
        
    name.append(f.find('h5',class_='text-center').text)
        
    designation.append(f.find('p',class_='small').text.strip().split('\n')[0])
    
    email.append(f.find('p',class_='mb-0').text)
    
    img.append("https://lhr.nu.edu.pk" + f.find('img', class_="card-img-top")['src'])

Extracting the Department for Each Faculty Member

In [6]:
computer_dept = soup_lhr.find('div',id='fsc')
computer_dept_faculty = computer_dept.descendants

for element in computer_dept_faculty:
    if element.name == 'h5' and 'text-center' in element.get('class', []):
            dept.append('Computing')

In [7]:
ee_dept = soup_lhr.find('div',id='ee')
ee_dept_faculty = ee_dept.descendants

for element in ee_dept_faculty:
    if element.name == 'h5' and 'text-center' in element.get('class', []):
            dept.append('Electrical Engineering')

In [8]:
cv_dept = soup_lhr.find('div',id='cv')
cv_dept_faculty = cv_dept.descendants

for element in cv_dept_faculty:
    if element.name == 'h5' and 'text-center' in element.get('class', []):
            dept.append('Civil Engineering')

In [9]:
fsm_dept = soup_lhr.find('div',id='fsm')
fsm_dept_faculty = fsm_dept.descendants

for element in fsm_dept_faculty:
    if element.name == 'h5' and 'text-center' in element.get('class', []):
            dept.append('Management')

In [10]:
ss_dept = soup_lhr.find('div',id='ss')
ss_dept_faculty = ss_dept.descendants

for element in ss_dept_faculty:
    if element.name == 'h5' and 'text-center' in element.get('class', []):
            dept.append('Sciences and Humanities')


In [11]:
# extracting links for each profile and extracting the highest education and extension  

for f in faculty_lhr:
    
    profile_URL = "https://lhr.nu.edu.pk" + f.find('a',class_='faculty-link')['href']
    profile_page = requests.get(profile_URL)
    profile_content = BeautifulSoup(profile_page.content, 'html.parser')
    profile = profile_content.find('div',class_='container')
    
    try:
        highest_education.append(profile.find('li').text)
        
    except AttributeError:
            highest_education.append('Not Available')
          
    try:
        extension.append(int(profile.find('span',class_="small").text.split()[-1][-3:]))
    
    except ValueError:
        extension.append(-1)

In [12]:
# creating the dataframe
lhr_df = pd.DataFrame({'ID': ID , 'Name': name , 'Designation': designation , 'HEC Approved PHD Supervisor': HEC_approved ,
                       'Highest Education': highest_education , 'Email' : email , 'Department': dept , 'Extension': extension ,
                       'ImageURL': img })
print(lhr_df)

lhr_df.to_csv('lhr.csv')

       ID                        Name         Designation  \
0    1238            Dr. Kashif Zafar     Professor & HOD   
1    4027              Dr. Aamir Wali           Professor   
2    4391    Dr. Asif Mahmood Gillani           Professor   
3    6113             Dr. Asma Naseer           Professor   
4    9540           Dr. Maryam Bashir           Professor   
..    ...                         ...                 ...   
213  5795           Mr. Razm-ul-Zafar            Lecturer   
214  6548            Ms. Rubab Arshad            Lecturer   
215  5337             Ms. Sarah Ahmad            Lecturer   
216  6363  Mr. Tasaduque Hussain Shah            Lecturer   
217  6567        Ms. Mahnoor Chaudhry  Lecturer (English)   

     HEC Approved PHD Supervisor  \
0                           True   
1                           True   
2                           True   
3                           True   
4                           True   
..                           ...   
213            